In [1]:
import os

if "CUDA_VISIBLE_DEVICES" not in os.environ:
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(
            gpu, True
        )  # Limiting the memory growth
    logical_gpus = tf.config.experimental.list_logical_devices("GPU")
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")

from pathlib import Path
import numpy as np
import pandas as pd

import os
import numpy as np
import random
import matplotlib.pyplot as plt 
import tensorflow as tf 
import psmiles 
import rdkit
from rdkit import Chem 
from rdkit.Chem import Draw 
import itertools

2025-02-04 14:49:26.495994: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-04 14:49:27.151031: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2025-02-04 14:49:28.520505: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14804 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:01:00.0, compute capability: 8.0


1 Physical GPUs, 1 Logical GPUs


In [2]:
import itertools

replacement_elements_0 = ['S','*-C=O','CS','C','CN','O','N']
replacement_elements_1 = ['[H]C(=O)O-*','FC(F)F','C(=O)Cl','NC(=O)']
replacement_elements_2 = ['(COOH)','(=C=C=0)','(=S)','(CF3)','(C(=O)Cl)','(NC(=O))']
replacement_elements_3 = ['(S)', '(=N)', '(=O)','(-C=0)','(#N)','(C#N)','(CS)','(SC)','(C)','(CN)','(O)','(N)']

end_groups_redox=[]

end_groups_redox += replacement_elements_0,replacement_elements_1,replacement_elements_2,replacement_elements_3

end_groups_redoxh=list(itertools.chain.from_iterable(end_groups_redox))

In [3]:
end_groups_redoxh += "C(=O)O", "C(=O)OC"

In [101]:
library=pd.read_json('/home/subhash/test/batterie_informatik/ne/Organic_battery_dataset.json')
l=library[library.Property=='V'].sort_values(by='value',ascending=False)
ps1,ps2=list(l.smiles)[1],list(l.smiles)[2]
#Taking the candidates with highest voltage in the dataset

In [187]:
k = dict(
    name=[
        "pn",
        "pn_with_o",
        "PE",
        "PVC",
        "PET",
        "PHB",
        "PLA",
        "PHA",
        "PP",
        "PS",
        "Poly_caprolactum",
        "PTFE",
    ],
    smiles=[
        ps1,
        "*-CC-*",
        "ClC(-*)C-*",
        "*-CCOC(=O)C1=CC=C(C=C1)C(=O)O-*",
        "CC(CC(*)=O)O*",
        "CC(-*)C(=O)O-*",
        "*OCC(*)=O",
        ps2,
        "CC(-*)C-*",
        "*-CC(-*)C1=CC=CC=C1",
        "*-CCCCCC(=O)O-*",
        "FC(F)(-*)C(F)(F)-*"
    ]

)
import rdkit
mols=[rdkit.Chem.MolFromSmiles(k) for k in k['smiles']]

In [7]:
side='&'
back='^'

from itertools import permutations

def generate_all_combinations(base_string, inserts):
    results = set()  # 
    # Single insertions
    for insert in inserts:
        for i in range(len(base_string) + 1):
            results.add(base_string[:i] + insert + base_string[i:])

    # Multiple insertions
    for r in range(2, len(inserts) + 1):
        for perm in permutations(inserts, r):
            current = [base_string]
            for insert in perm:
                new_combinations = []
                for s in current:
                    for i in range(len(s) + 1):
                        new_combinations.append(s[:i] + insert + s[i:])
                current = new_combinations
            results.update(current) 
    
    return sorted(results) 

In [8]:
import pandas as pd
import numpy as np
import itertools
inserts=[back,side]

for name,smile in zip(k['name'],k['smiles']):
    combinations=generate_all_combinations(smile,inserts)
    backbone_sidechain = []
    for init in combinations:
        if '^' in init and '&' in init:
            for side,back in itertools.permutations(end_groups_redoxh,2):
                for num_back in range(1,3):
                    temp_back= init.replace("^", back * num_back)
                    for num_side in range(1,3):
                        repl = "(" +  num_side*side + ")"
                        temp_side = temp_back.replace("&", repl)
                        backbone_sidechain.append(temp_side)
        elif '&' in init and '^' not in init:
            for side in end_groups_redoxh:
                for num_side in range(1,3):
                    repl = "(" +  num_side*side + ")"
                    temp_side = init.replace("&", repl)
                    backbone_sidechain.append(temp_side)
        elif '&' not in init and '^' in init:
            for back in end_groups_redoxh:
                for num_back in range(1,3):
                    temp_back= init.replace("^", back * num_back)
                    backbone_sidechain.append(temp_back)
        else:
            continue
    df=pd.DataFrame(backbone_sidechain)
    df=df.drop_duplicates(subset=0,ignore_index=True)
    df.to_parquet(f'{name}_bp.parquet')

In [ ]:
from itertools import combinations, product
def replace_with_elements(smiles, elements):
    length = len(smiles)
    a=[]
    # Replace at one index
    for i in range(length):
        modified_smiles = list(smiles)  # Convert to list for mutability
        modified_smiles[i] = '0'  # Replace character at index i
        a.append(generate_replacements(''.join(modified_smiles), elements))

    # Replace at two indices
    for i, j in combinations(range(length), 2):
        modified_smiles = list(smiles)
        modified_smiles[i] = '0'  # Replace character at index i
        modified_smiles[j] = '0'  # Replace character at index j
        a.append(generate_replacements(''.join(modified_smiles), elements))

    # Replace at three indices
    for i, j, k in combinations(range(length), 3):
        modified_smiles = list(smiles)
        modified_smiles[i] = '0'  # Replace character at index i
        modified_smiles[j] = '0'  # Replace character at index j
        modified_smiles[k] = '0'  # Replace character at indePTFE_rep3x k
        a.append(generate_replacements(''.join(modified_smiles), elements))
    
    return a

def generate_replacements(modified_smiles, elements):
    length = modified_smiles.count('0')
    f=[]
    # Generate all possible replacements for '0'
    for replacement in product(elements, repeat=length):
        result = list(modified_smiles)
        
        # Replace '0's with the corresponding element from replacement tuple
        replacement_index = 0
        for i in range(len(result)):
            if result[i] == '0':
                result[i] = replacement[replacement_index]
                replacement_index += 1
        
        f.append(''.join(result))  # Print the final modified string
    return f

a3=replace_with_elements(smiles_string, replacement_elements_3)
merged = list(set(list(itertools.chain.from_iterable(a3))))
m=pd.DataFrame(merged)
m.to_parquet(f'{name}_{replacement_elements_3}.parquet')